In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# remover comentarios si sed quiere instalar la libreria 
# publicada en pipy
#%pip uninstall azure-datalake-utils -y
#%pip install -i https://test.pypi.org/simple/ --extra-index-url=https://pypi.org/simple azure-datalake-utils==0.2.0.dev23
#%pip show azure-datalake-utils

In [3]:
import logging
import sys
sys.path.append('../')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

Asegurar crear primero el archivo `.env` con las siguientes variables de ambiente:
```
datalake="nombre_datalake_cuenta"
datalake_key="xxxkey_datalake_cuenta_zxccxxcx"
````

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from centraal_dataframework.resources import datalake
info = datalake.read_csv("test/test_framework.csv", sep="|")

In [25]:
import great_expectations as gx

context = gx.get_context(mode = "ephemeral")
datasource = context.sources.add_pandas(name="my_pandas_datasource")
data_asset = datasource.add_dataframe_asset(name="test")
my_batch_request = data_asset.build_batch_request(dataframe=info)

In [26]:
context.add_or_update_expectation_suite("my_expectation_suite")
validator = context.get_validator(
    batch_request=my_batch_request,
    expectation_suite_name="my_expectation_suite",
)
validator.expect_column_values_to_not_be_null("Producto")
validator.save_expectation_suite()

/home/german/repos/centraal-dataframework/.venv/lib/python3.8/site-packages/great_expectations/expectations/expectation.py:1481: UserWarning: `result_format` configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(


Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

In [27]:
checkpoint = context.add_or_update_checkpoint(
    name="my_checkpoint",
    validations=[
        {
            "batch_request": my_batch_request,
            "expectation_suite_name": "my_expectation_suite",
        },
    ],
)
checkpoint_result = checkpoint.run()

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

In [28]:
context.view_validation_result(checkpoint_result)

Start : This command cannot be run due to the error: The system cannot find the file specified.
At line:1 char:1
+ Start "file:///tmp/tmpd3pptoh6/validations/my_expectation_suite/__non ...
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidOperation: (:) [Start-Process], InvalidOperationException
    + FullyQualifiedErrorId : InvalidOperationException,Microsoft.PowerShell.Commands.StartProcessCommand
 


Crear el modulo con la logica, en la siguiente celda se configura una logica de pandas.

In [5]:
%%writefile logica_pandas.py
"""Prueba."""
from centraal_dataframework.tasks import task


@task
def hello_datalake(datalake, logger):
    """Saluda al datalake."""
    source = datalake.read_csv("test/test_framework.csv", sep="|")
    logger.info(source.head(1))
    logger.info("hello there, iam a task")


Overwriting logica_pandas.py


O puedes crear una estructura mas compleja de modulos. en la siguiente celda se crea un modulo dentro de otra carpeta y se usa un decorador para poder usar great expectations.

In [6]:
%mkdir modulo

mkdir: cannot create directory ‘modulo’: File exists


In [7]:
%%writefile modulo/__init__.py
"""Prueba."""

Overwriting modulo/__init__.py


In [8]:
%%writefile modulo/logica_gx.py
"""Prueba gx."""
from centraal_dataframework.tasks import task_dq


@task_dq
def hello_gx(datalake, datasource, logger):
    """Saluda al gx."""
    source = datalake.read_csv("test/test_framework.csv", sep="|")
    logger.info(source.head(1))
    logger.info("hello there, iam a task")


@task_dq(source_name="logica_negocio")
def hello_gx_negocio(datalake, datasource, logger):
    """Saluda al gx con logica."""
    source = datalake.read_csv("test/test_framework.csv", sep="|")
    logger.info(source.head(1))
    logger.info("hello there, iam a task")


Overwriting modulo/logica_gx.py


A continuación un se muestra el contenido que debe tener `function_app.py`

In [9]:
# contenido de function_app.py
import azure.functions as func
from centraal_dataframework.blueprints import framework
# se deben importar las funciones, ejemplo todas las funciones dentro
# de uno
from logica_pandas import *
# o se pueden importar uno a uno
from modulo.logica_gx import hello_gx
from modulo.logica_gx import hello_gx_negocio

app = func.FunctionApp()
# se agrega el framework:
app.register_functions(framework)

Realizar test dentro del notebook.

In [10]:
from azure.functions import QueueMessage

queue_trigger = app.get_functions()[1].get_user_function()
# simular la ejecucion de la tarea.
queue_trigger(QueueMessage(body = bytes("otra_cosa",  "utf-8")))

se presento error en otra_cosa
Traceback (most recent call last):
  File "/home/german/repos/centraal-dataframework/centraal_dataframework/blueprints.py", line 62, in execute_tasks_inqueue
    func_to_execute()
TypeError: 'NoneType' object is not callable


ErrorEnTarea: Se encontro un error en otra_cosa.

In [ ]:
# simular la ejecucion de la tarea.
queue_trigger(QueueMessage(body = bytes("otra_cosa",  "utf-8")))